# The GUI is based on V3 of coode that receives data from from FEDs and sends TTL to TDT system.


In [ ]:
import tkinter as tk
from tkinter import scrolledtext
import threading
import time
import serial
import RPi.GPIO as GPIO
import os
import sys
import logging
import datetime

# Configure logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s [%(levelname)s] %(message)s', stream=sys.stdout)

# GPIO setup (for reference from your existing code)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins per device (same as your existing setup)
gpio_pins_per_device = {
    'Port 1': { ... },
    'Port 2': { ... },
    'Port 3': { ... },
    'Port 4': { ... }
}

# GUI class
class FED3MonitorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("FED3 Monitor")
        
        # Create panels for each port
        self.port_frames = {}
        for port in ['Port 1', 'Port 2', 'Port 3', 'Port 4']:
            frame = tk.LabelFrame(root, text=port, padx=10, pady=10)
            frame.pack(padx=10, pady=10, side=tk.LEFT, expand=True, fill=tk.BOTH)
            label_status = tk.Label(frame, text="Status: Initializing...", font=("Helvetica", 12))
            label_status.pack()
            text_area = scrolledtext.ScrolledText(frame, height=10, width=30, wrap=tk.WORD)
            text_area.pack(padx=10, pady=10)
            text_area.insert(tk.END, f"Waiting for data from {port}...\n")
            self.port_frames[port] = {
                "frame": frame,
                "status": label_status,
                "text_area": text_area
            }

        # Start a background thread to handle the FED3 devices
        self.start_device_threads()

    def update_status(self, port, status):
        # Update the status label for the corresponding port
        self.port_frames[port]["status"].config(text=f"Status: {status}")

    def update_text_area(self, port, data):
        # Append data to the scrolled text area for the corresponding port
        text_area = self.port_frames[port]["text_area"]
        text_area.insert(tk.END, data + '\n')
        text_area.see(tk.END)  # Auto-scroll to the end

    def start_device_threads(self):
        # Start the threads to read from FED3 devices
        device_mappings = get_device_mappings_by_usb_port()
        if not device_mappings:
            logging.error("No FED3 devices found.")
            return

        for mapping in device_mappings:
            port_identifier = mapping['port_identifier']
            serial_port = mapping['serial_port']
            gpio_pins = gpio_pins_per_device.get(port_identifier)

            threading.Thread(target=self.read_from_fed, args=(serial_port, port_identifier, gpio_pins)).start()

    def read_from_fed(self, serial_port, port_identifier, gpio_pins):
        # Open the serial port and read data in the background thread
        try:
            ser = serial.Serial(serial_port, 115200, timeout=1)
            logging.info(f"Opened serial port {serial_port} for {port_identifier}")
            self.update_status(port_identifier, "Ready!")
        except serial.SerialException as e:
            logging.error(f"Could not open serial port {serial_port}: {e}")
            self.update_status(port_identifier, "Error")
            return

        try:
            while True:
                line = ser.readline().decode('utf-8').strip()
                if line:
                    data_list = line.split(",")
                    self.update_text_area(port_identifier, f"Data: {data_list}")
                    # You can process the events here as well, similar to your existing code
                else:
                    time.sleep(0.1)  # Prevent CPU hogging
        except Exception as e:
            logging.error(f"Error reading from serial port {serial_port}: {e}")
        finally:
            ser.close()

def get_device_mappings_by_usb_port():
    # Simulating the device mappings based on your existing function
    # This will need the same implementation logic as your original function
    device_mappings = [
        {'serial_port': '/dev/ttyUSB0', 'port_identifier': 'Port 1'},
        {'serial_port': '/dev/ttyUSB1', 'port_identifier': 'Port 2'},
        {'serial_port': '/dev/ttyUSB2', 'port_identifier': 'Port 3'},
        {'serial_port': '/dev/ttyUSB3', 'port_identifier': 'Port 4'},
    ]
    return device_mappings

if __name__ == "__main__":
    root = tk.Tk()
    app = FED3MonitorApp(root)
    root.mainloop()
